# 入门云原生AI - 2. 分布式mnist

在深度学习世界里，算力即能力。对于网络层数较深的模型，和非常大的数据集，单机的GPU计算能力已经无法满足我们的需求。这时候我们需要分布式的深度学习解决方案。而当任务从单机发展到分布式，我们有paramter server和基于MPI的horovod这些方案解决代码层面的问题，可以帮助我们轻松将单机训练的代码变成支持分布式训练的代码。但是在运维层面，随着部署拓扑变得复杂，实例个数变多，任务数量剧增，如何高效部署，管理，调度，可重复运行分布式任务，是我们无法回避的严峻问题。

而随着容器的出现和Kubernetes的诞生，带来了天然的可移植性和异构系统的调度管理能力，帮助我们更加方便快捷地运维和管理。但是正如软件工程行业的铁律「没有银弹」，Kubernetes提供了丰富而强扩展能力的编排定义模式, 但随之而来的是一定的学习成本和使用复杂度。 对于数据科学家来说，更应该专注于深度学习的工作本身，通过更高抽象的方式下发和管理查看任务。

而arena的诞生就是为了解决这个问题， arena支持一系列的命令，可以通过简单的声明方式描述下发任务的拓扑信息，例如MPI任务中的Workers数量，Paramter server模式中PS和Workers的数量，GPU数量，是否开启Tensorboard 等，以及监控，日志等能力。
接下来我们从一个分布式mnist示例开始，介绍如何通过arena提交，运维，管理一个分布式训练任务。
在这个示例中，我们将演示：

* 下载并准备数据
* 利用Arena提交单机训练任务,并且查看训练任务状态和日志，监控GPU信息
* 通过TensorBoard查看训练任务

> 前提：请先完成文档中的[共享存储配置]()

1.下载TensorFlow样例源代码到${HOME}/models目录

In [10]:
! git clone https://code.aliyun.com/xiaozhou/tensorflow-sample-code.git ${HOME}/models/tensorflow-sample-code

Cloning into '/root/models/tensorflow-sample-code'...
remote: Enumerating objects: 242, done.
remote: Counting objects: 100% (242/242), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 242 (delta 93), reused 242 (delta 93)
Receiving objects: 100% (242/242), 11.25 MiB | 0 bytes/s, done.
Resolving deltas: 100% (93/93), done.
Checking connectivity... done.


2.下载mnist数据到${HOME}/dataset/mnist

In [23]:
! mkdir -p ${HOME}/dataset/mnist && \
  cd ${HOME}/dataset/mnist && \
  curl -O https://code.aliyun.com/xiaozhou/tensorflow-sample-code/raw/master/data/t10k-images-idx3-ubyte.gz && \
  curl -O https://code.aliyun.com/xiaozhou/tensorflow-sample-code/raw/master/data/t10k-labels-idx1-ubyte.gz && \
  curl -O https://code.aliyun.com/xiaozhou/tensorflow-sample-code/raw/master/data/train-images-idx3-ubyte.gz && \
  curl -O https://code.aliyun.com/xiaozhou/tensorflow-sample-code/raw/master/data/train-labels-idx1-ubyte.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1610k    0 1610k    0     0  2240k      0 --:--:-- --:--:-- --:--:-- 2242k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4542    0  4542    0     0  12438      0 --:--:-- --:--:-- --:--:-- 12409
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9680k    0 9680k    0     0  11.2M      0 --:--:-- --:--:-- --:--:-- 11.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 28881    0 28881    0     0  70105      0 --:--:-- --:--:-- --:--:-- 70270


3.检查可用GPU资源

In [4]:
! arena top node

NAME                                   IPADDRESS      ROLE    GPU(Total)  GPU(Allocated)
cn-zhangjiakou.i-8vb2knpxzlk449e7lugx  192.168.0.209  <none>  1           0
cn-zhangjiakou.i-8vb2knpxzlk449e7lugy  192.168.0.210  <none>  1           0
cn-zhangjiakou.i-8vb2knpxzlk449e7lugz  192.168.0.208  <none>  1           0
cn-zhangjiakou.i-8vb7yuo831zjzijo9sdw  192.168.0.205  master  0           0
cn-zhangjiakou.i-8vbezxqzueo7662i0dbq  192.168.0.204  master  0           0
cn-zhangjiakou.i-8vbezxqzueo7681j4fav  192.168.0.206  master  0           0
-----------------------------------------------------------------------------------------
Allocated/Total GPUs In Cluster:
0/3 (0%)  


4.通过Arena提交训练任务
这里`training-data`在配置[共享存储时]()创建.  
`--data=training-data:/training` 将其映射到训练任务的 `/training` 目录。  
* `/training` 目录下的子目录`/training/models/tensorflow-sample-code` 就是步骤1拷贝源代码的位置，
* `/training` 目录下的子目录`/training/dataset/mnist` 就是步骤2下载数据的位置。

和单机任务不同，TF Paramter server模式的分布式训练任务，需要指定Paramter Server和Worker的数量， 示例中我们指定为1个Paramter Server，2个Workers，两个workers分别使用1个GPU。

In [33]:
# 提交一个分布式训练任务
!arena submit tf \
             --name=distribute-mnist \
             --ps=1 \
             --workers=2 \
             --gpus=1 \
             --data=training-data:/training \
             --image=tensorflow/tensorflow:1.11.0-gpu-py3 \
             "python /training/models/tensorflow-sample-code/tfjob/docker/v1alpha2/distributed-mnist/main.py --max_steps 10000 --data_dir /training/dataset/mnist"

configmap/distribute-mnist-tfjob created
configmap/distribute-mnist-tfjob labeled
service/distribute-mnist-tensorboard created
deployment.extensions/distribute-mnist-tensorboard created
tfjob.kubeflow.org/distribute-mnist created
INFO[0003] The Job distribute-mnist has been submitted successfully 
INFO[0003] You can run `arena get distribute-mnist --type tfjob` to check the job status 


> 完整参数可以参考[命令行文档](https://github.com/kubeflow/arena/blob/master/docs/cli/arena_submit_tfjob.md)

5. 检查模型训练状态，当任务状态从`Pending`转为`Running`后就可以查看日志和GPU使用率了。这里`-e`为了方便检查任务`Pending`的原因。
我们可以通过get命令查看到这个任务的容器分布，示例中有1个ps容器和2个workers容器。

In [27]:
! arena get distribute-mnist -e

STATUS: RUNNING
NAMESPACE: default
TRAINING DURATION: 2m

NAME              STATUS   TRAINER  AGE  INSTANCE                   NODE
distribute-mnist  RUNNING  TFJOB    2m   distribute-mnist-ps-0      192.168.0.208
distribute-mnist  RUNNING  TFJOB    2m   distribute-mnist-worker-0  192.168.0.210
distribute-mnist  RUNNING  TFJOB    2m   distribute-mnist-worker-1  192.168.0.209

Your tensorboard will be available on:
192.168.0.206:32032   

Events: 
No events for pending pod


6.实时检查日志

In [28]:
# get the job logs
! arena logs -f --tail=50 distribute-mnist

2019-02-22T13:44:03.262560773Z Adding run metadata for 1999
2019-02-22T13:44:03.262563328Z Accuracy at step 2000: 0.9801
2019-02-22T13:44:03.262565685Z Accuracy at step 2010: 0.9772
2019-02-22T13:44:03.262568164Z Accuracy at step 2020: 0.9769
2019-02-22T13:44:03.262570601Z Accuracy at step 2030: 0.9777
2019-02-22T13:44:03.262573095Z Accuracy at step 2040: 0.9803
2019-02-22T13:44:03.262575451Z Accuracy at step 2050: 0.9785
2019-02-22T13:44:03.26257794Z Accuracy at step 2060: 0.9775
2019-02-22T13:44:03.2625803Z Accuracy at step 2070: 0.978
2019-02-22T13:44:03.262582851Z Accuracy at step 2080: 0.978
2019-02-22T13:44:03.26258519Z Accuracy at step 2090: 0.9789
2019-02-22T13:44:03.262587707Z Adding run metadata for 2099
2019-02-22T13:44:03.262590064Z Accuracy at step 2100: 0.9781
2019-02-22T13:44:03.262592509Z Accuracy at step 2110: 0.9778
2019-02-22T13:44:03.262594844Z Accuracy at step 2120: 0.977
2019-02-22T13:44:03.262597213Z Accuracy at step 2130: 0.9782
2019-02-22T13:44:03.26259955Z Acc

7.查看实时训练的GPU使用情况

In [31]:
! arena top job

NAME              GPU(Requests)  GPU(Allocated)  STATUS   TRAINER  AGE  NODE
distribute-mnist  2              2               RUNNING  tfjob    3m   192.168.0.210


Total Allocated GPUs of Training Job:
2   

Total Requested GPUs of Training Job:
2   


8.通过TensorBoard查看训练趋势。您可以使用 `192.168.1.117:30670` 访问 Tensorboard。如果您通过笔记本电脑无法直接访问 Tensorboard，可以考虑使用 `sshuttle`。例如：`sshuttle -r root@41.82.59.51 192.168.0.0/16`。其中`41.82.59.51`为集群内某个节点的外网IP，且该外网IP可以通过ssh访问。

In [35]:
! arena get distribute-mnist

STATUS: RUNNING
NAMESPACE: default
TRAINING DURATION: 1m

NAME              STATUS     TRAINER  AGE  INSTANCE                   NODE
distribute-mnist  RUNNING    TFJOB    1m   distribute-mnist-ps-0      192.168.0.208
distribute-mnist  RUNNING    TFJOB    1m   distribute-mnist-worker-0  192.168.0.210
distribute-mnist  SUCCEEDED  TFJOB    1m   distribute-mnist-worker-1  N/A

Your tensorboard will be available on:
192.168.0.206:32684   


![](2-tensorboard.jpg)

9.删除已经完成的任务

In [36]:
! arena delete distribute-mnist

service "distribute-mnist-tensorboard" deleted
deployment.extensions "distribute-mnist-tensorboard" deleted
tfjob.kubeflow.org "distribute-mnist" deleted
configmap "distribute-mnist-tfjob" deleted
INFO[0004] The Job distribute-mnist has been deleted successfully 


恭喜！您已经使用 `arena` 成功运行了训练作业，而且还能轻松检查 Tensorboard。

总结，希望您通过本次演示了解：
1. 如何准备代码和数据，并将其放入数据卷中
2. 如何在训练任务中引用数据卷，并且使用其中的代码和数据
3. 如何利用arena管理您的分布式训练任务。